In [129]:
import pandas as pd

requests_df = pd.read_csv("../data/cleaned_data/cleaned_ATL311_2022.csv")        # data file
geo_df = pd.read_csv("../data/cleaned_data/geocode_cache.csv")         # cache file


C:\Users\shrey\AppData\Local\Temp\ipykernel_41848\3771349905.py:4: DtypeWarning: Columns (1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  geo_df = pd.read_csv("../data/cleaned_data/geocode_cache.csv")         # cache file


In [130]:
def normalize_address(addr):
    if pd.isna(addr):
        return None
    addr = str(addr).strip()
    addr = addr.strip('"').strip("'")
    addr = addr.replace(" ,", ",")
    addr = " ".join(addr.split())
    return addr


In [131]:
requests_df["address_norm"] = requests_df["Address"].apply(normalize_address)
geo_df["address_norm"] = geo_df["address"].apply(normalize_address)


In [132]:
merged_df = requests_df.merge(
    geo_df[["address_norm", "lat", "lng", "ok"]],
    on="address_norm",
    how="left"
)


In [133]:
matched_df = merged_df[merged_df["lat"].notna()]
unmatched_df = merged_df[merged_df["lat"].isna()]


In [134]:
print(f"Matched: {len(matched_df)}")
print(f"Unmatched: {len(unmatched_df)}")

print(unmatched_df["Address"].unique())


Matched: 52088
Unmatched: 419
['1849 Venetian Dr  , Atlanta , 30311' '329 Hill St  , Atlanta, 30312'
 '3554 Tuxedo Park  , Atlanta , 30305'
 '200 Stanhope Cir  , Atlanta , 30314'
 '847 Commonwealth Ave  , ATLANTA, 30312'
 '2284 Cottage grove ave , Atlanta  , 30317'
 '1161 Eggleston St  , ATLANTA, 30310'
 '2363 Montview Dr  , Atlanta, 30305'
 '4510 Northside Drive  , Atlanta, 30327'
 '850 Fairfield Rd  , Atlanta , 30327'
 '655 Magnolia St  , Atlanta , 30314' '958 SMITH ST SW   , Atlanta, 30310'
 '535 MILAM AVE SW  , Atlanta, 30315'
 '505 W Ontario Ave  , Atlanta , 30310'
 '557 Commercial Ave  , Atlanta , 30318'
 '361 PHARR RD NE  , Atlanta, 30305' '181 Torrance st  , Atlanta , 30314'
 '1355 PEACHTREE ST NE STE 1125 , Atlanta  , 30309'
 '975 Eisenhower Rd   , Atlanta , 30354'
 '3269 Benjamin E Mays  , Atlanta , 30311'
 '3253 Jamaica Rd  , Atlanta , 30318' '3261 Jamaica Rd  , Atlanta , 30318'
 '1104 OAKLAND DR SW  , Atlanta, 30310'
 '3611 Tuxedo Ct  , Atlanta , 30305'
 '152 Vine Street SW

In [89]:
orig_unmatched = unmatched_df.copy()
print(len(orig_unmatched))


368


In [90]:
import re
import pandas as pd

def extract_street_debug(addr):
    if pd.isna(addr):
        return None

    original = addr

    addr = str(addr).lower()

    # remove quoted junk
    addr = re.sub(r'"[^"]*"', '', addr)

    # remove "intersection of"
    addr = addr.replace("intersection of", "")

    # split on city names
    addr = re.split(r'\batlanta\b', addr)[0]

    # normalize
    addr = " ".join(addr.split()).strip(" ,")

    return addr


In [91]:
geo_df["street_only"] = geo_df["address_norm"].str.lower().apply(extract_street_debug)


In [92]:
def debug_fallback_match(addr, geo_df):
    street = extract_street_debug(addr)

    if not street:
        return {
            "extracted_street": None,
            "matched_cache_address": None,
            "lat": None,
            "lng": None,
            "ok": False
        }

    for _, row in geo_df.iterrows():
        cached_street = row["street_only"]
        if not cached_street:
            continue

        if street in cached_street or cached_street in street:
            return {
                "extracted_street": street,
                "matched_cache_address": row["address"],
                "lat": row["lat"],
                "lng": row["lng"],
                "ok": True
            }

    return {
        "extracted_street": street,
        "matched_cache_address": None,
        "lat": None,
        "lng": None,
        "ok": False
    }


In [93]:
debug_rows = []

for _, row in orig_unmatched.iterrows():
    result = debug_fallback_match(row["Address"], geo_df)

    debug_rows.append({
        "original_address": row["Address"],
        "extracted_street": result["extracted_street"],
        "matched_cache_address": result["matched_cache_address"],
        "lat": result["lat"],
        "lng": result["lng"],
        "ok": result["ok"]
    })

debug_df = pd.DataFrame(debug_rows)


KeyboardInterrupt: 

In [79]:
debug_df


,original_address,extracted_street,matched_cache_address,lat,lng,ok
0,"630 Burke Rd NE , Atlanta , 30305",630 burke rd ne,"630 Burke Rd NE, Atlanta , 30305",33.836186,-84.366256,True
1,"447 PAINES AVE NW , ATLANTA, 30318",447 paines ave nw,"447 PAINES AVE NW , ATLANTA, 30318",33.766950,-84.414542,True
2,"2 Blanche ST NW , Atlanta, 30318",2 blanche st nw,"2 Blanche ST NW , Atlanta, 30318",33.771595,-84.444761,True
3,"2400 Campbellton Road SW , Atlanta , 30311",2400 campbellton road sw,"2400 Campbellton Road SW , Atlanta, 30311",33.703212,-84.465243,True
4,"1562 Mayflower Ave SW, Atlanta , 30311",1562 mayflower ave sw,"1562 Mayflower Ave SW, Atlanta , 30311",33.727612,-84.439209,True
5,"334 GLENCASTLE DR NW , ATLANTA, 30327-4826",334 glencastle dr nw,"334 GLENCASTLE DR NW , ATLANTA, 30327-4826",33.900684,-84.398466,True
6,"1246 Ladd St SW , Atlanta, 30310",1246 ladd st sw,"1246 Ladd St SW , Atlanta, 30310",33.727129,-84.428708,True
7,Intersection of Collier Rd and Springlake Dr N...,collier rd and springlake dr nw,Intersection of Collier Rd and Springlake Dr N...,33.809459,-84.411387,True
8,"""Hosea L Williams Dr SE"", Whitefoord Ave SE, A...",whitefoord ave se,"104 WHITEFOORD AVE SE, ATLANTA, GA, 30317",33.750573,-84.341983,True
9,"908 Emerson Ave SE , Atlanta, 30316",908 emerson ave se,"908 Emerson Ave SE , Atlanta, 30316",33.733064,-84.357840,True


In [127]:
matched_now = merged_df[merged_df["lat"].notna()]
unmatched_now = merged_df[merged_df["lat"].isna()]

print(f"Total records: {len(merged_df)}")
print(f"Matched now: {len(matched_now)}")
print(f"Still unmatched: {len(unmatched_now)}")


Total records: 104884
Matched now: 104417
Still unmatched: 467


In [111]:
bad_addresses = [
    "1126 Euclid Ave NE , 30307",
    "4510 Northside Drive , Atlanta, 30327",
    "7141 CAVENDER DR SW , FAIRBURN, 30213",
    "2147 Burroughs ave , ATLANTA, 30315", 
    "245 Richardson Rd NW  , Atlanta, 30314",
    "1470 Bouldercrest Rd SE , ATLANTA, 30316"
]


In [128]:
merged_df.to_csv(
    "../data/cleaned_data/atl311_2021_merged_with_geocodes.csv",
    index=False
)

print("merged_df saved successfully ✅")


merged_df saved successfully ✅
